In [5]:
import sys
!{sys.executable} -m pip install pandas-profiling
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree
import pandas_profiling

    100% |████████████████████████████████| 133kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 614kB 3.9MB/s ta 0:00:01


    100% |████████████████████████████████| 225kB 3.4MB/s ta 0:00:01
  Stored in directory: /Users/Inceptive/Library/Caches/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Stored in directory: /Users/Inceptive/Library/Caches/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Stored in directory: /Users/Inceptive/Library/Caches/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
Successfully built pandas-profiling htmlmin confuse


In [2]:
df = pd.read_csv('SpotifyAudioFeaturesApril2019 copy.csv')

In [3]:
df.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


In [6]:
df.profile_report()

In [34]:
# Now to Make a Function to sum all this up and try different clustering models

def find_nearest_songs(df, number_of_songs):
    # remove categoricals
    df_numerics =  df.drop(columns=['track_id', 'track_name', 'artist_name'])
    
    # Scale Data To Cluster More Accurately, and fit clustering model
    df_scaled = StandardScaler().fit_transform(df_numerics)
    df_modeled = KDTree(df_scaled)
    
    # Querying the model for the 15 Nearest Neighbors
    dist, ind = df_modeled.query(df_scaled, k=16)
    
    # Putting the Results into a Dataframe
    dist_df = pd.DataFrame(dist)
    
    # Calculating the Distances
    scores = (1 - ((dist - dist.min()) / (dist.max() - dist.min()))) * 100
    
    # Creating A New Dataframe for the Distances
    columns = ['Searched_Song', 'Nearest_Song1', 'Nearest_Song2', 'Nearest_Song3', 'Nearest_Song4',
               'Nearest_Song5', 'Nearest_Song6', 'Nearest_Song7', 'Nearest_Song8', 'Nearest_Song9',
               'Nearest_Song10', 'Nearest_Song11', 'Nearest_Song12', 'Nearest_Song13', 'Nearest_Song14',
               'Nearest_Song15']
    dist_score = pd.DataFrame(scores.tolist(), columns = columns)
    
    # An Array of all indices of the nearest neighbors
    ind[:16]
    
    # Making an array of the Track IDs
    song_ids = np.array(df.track_id)
    
    # A function that creates list of the each song with its nearest neighbors
    def find_similars(song_ids, ind):
        similars = []
        for row in ind:
            ids = [song_ids[i] for i in row]
            similars.append(ids)

        return similars 
    
    # using the above function
    nearest_neighbors = find_similars(song_ids, ind)
    
    # putting the results into a dataframe
    nearest_neighbors_df = pd.DataFrame(nearest_neighbors, columns=columns)
    
    return nearest_neighbors_df

In [35]:
# this takes a good two to three minutes to process
find_nearest_songs(df)

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10,Nearest_Song11,Nearest_Song12,Nearest_Song13,Nearest_Song14,Nearest_Song15
0,2RM4jf1Xa9zPgMGRDiht8O,2ogGsvbGjJz71Ppc9mTExA,56GWUZPELOP1nK4hQwfVVV,02s1Voowwhr0qTSOrMVEXk,1rG9U7m2wqM0AVmZeIsdtz,3UDOPqusSZZ7NqyWyMMk50,7oDuAatXvqWD8DAa87KRJ2,0MiAP4EQGIuikH23RTyP53,2lD4dQpcac8H2Rq5nttJJh,2MwIkkJujEmXa7o9Nc6HeI,06oDi0OztS33sxfYtxJPO2,0w6GW2cBPbK941JkxbuvZn,3NNyX4gtwb8JGjoRfL0bxY,1yVYXKx1MT9wlk3KjNOCIN,6yoM6KoDpWXKIWo9UnZ3Ef,4N6vSANRRLq99sY9f01mrA
1,1tHDG53xJNGsItRA3vfVgs,76wqmxKVKXradVxmfGpZfX,0x1MJ34QQHGK9nTmOdSe33,0D8Qknpc7ZtnOU4pEwXGos,3eUyRzsgH4Vjraw3oVqVDg,51jT8cWCrANwOvDkzqKpF4,7vBFNgj6XAYYcp6pKuvjp2,4uO9h2EWJqr1ViWEON957k,1Dfwu69pIy52gVUzbJ6Bwk,1SE0FG6lc3MMZoHj65DTVq,7A5WzjLKRSglnolIcKDJHm,7KUTK7cBE2bPO9tEFnMKlD,2qC2KmT96fgWzVDagTVKTj,6nARENOM1f4ENcE7MMmALN,77qiFgmXvyFFeI0iyHQxoM,72s4ePV5YfZCnBrG5QFQIO
2,6Wosx2euFPMT14UXiWudMy,6Wosx2euFPMT14UXiWudMy,6MdsMUihzXs34YqYjXy01a,2BNQpd87nSv475zdww9K0Z,54JNbExxKpEXUqeSc63Xwd,6RMMWRA9UY9D8DlPNSUytf,76b1sn3qXpA4PhtSaIHoK7,3z7AceXFwECHGH26FOy3u8,6TXAjof8V0Gr4zFoY00zHR,5puKxbSkahwtI2XJR1xZgr,5hZlzFaN6L8yw1a0VdWpzF,3idyB7eZlIhTxp1baHvjlW,72DdM3q1npvvRUoa1YQEOd,6sOlZklNPNnXojbXKn8Wfe,4bkJKBNgPESM7fH63Bra0I,79bhY6Rl8q2VJRHAbWNCGl
3,3J2Jpw61sO7l6Hc7qdYV91,1jFL774GMnBl5Esa9SN4rL,38oxSM8IHLxt43QV7amf4u,01qc7HHvJdcNviyizhuCSI,17ztPex4W8AT6A6eiNipJ0,4EMfeNehRq5ylzAnjyeyXD,0a7U85BtD3pOKrLMvmoTg0,0xoanCCNnBPaHed0C6VhsV,5KPBYV9FSX88T65iR0zt9a,4Yld8cz0LA6a8n8tEDEsfw,09sEGuu6BQPLNCi1uoFTpa,3ZHjVjAhfsVBZKcCznUsWr,7rzdrkDwzhxBMJ58TPMf2m,50B1ZAroYy7eJyQWqNsQTX,21DU83QG4jB4mQKh76X32h,3ZJuGqDZ9iU9hMPnB9rsju
4,2jbYvQCyPgX3CdmAzeVeuS,6cjG5e09jR9cmwrNWf33rN,3Sxt87DUcTwIocwQUgnOaB,40En76NSz9PLSXcMMED8OX,3dTwRbYD0xu6F0tDTaQMdF,1aTirUVIiRD28x8KkXtcjx,0ttUWrdZ5gCPyGxulSgL5q,0rNoxCiFdTazhLNv2pHb5c,52zEbsrYSVZ8Vj5XNI8iCg,3HlPpFnbHsnOzrg0X9VOF7,3WaffItZ24L8YP5DMYMESj,63MyqXy8yCu8WTIDvd9JeI,5kvv2bQ9r8SZoOO1VkCHY1,3mOr32iRqSnJMZRlIFsP2h,2XafhjQX0a7BefPhkyxQ9P,5nbW9Dt9twjfZx8ukVUKt3
5,26Y1lX7ZOpw9Ql3gGAlqLK,37abXzUmuQ7QvAzdjfewQN,6zZ5xX97VxwqXdmsPvlc2S,5Cbg9k7PFZlAWk7z3EaoBh,21TDTv7Wq8t8grLgDVeWnw,1FkFhI9SvQBQvha6LTSbiH,2Ox1C7ngWDX81PLEhxzxNM,3nWJtWzWfNZDYBUNbq29lf,4xNkEyoCniyoOoVawcGzMR,1YkvO7fAA73996Na8CyHMv,7HOJOY3gZ1b20nvlG1OmhD,6OD02jC5WRugIsVCkMqWt1,7fE6KIi7eZthmX1mvX48Ej,5Iwjb8jUwDU06jkWrHUdj2,5aH5Q8ULg2dVsbIKgYZc1r,2u7iOovuZuLf4tZMevgq1C
6,5eIyK73BrxHLnly4F9PWqg,619DpDoIozEoT2VI8eGchi,3k6ryJkZHMlXNkbadxj7lr,1g7DVPLJl7adbGDsJjmr1W,5ZQCFtQuQc5pX0w0sbLeo7,3RyyRov5PZ1IXSA4Iv32Qg,1D9Eu8yENZ4v72xpShanBB,0WjrbhsdpYJx5x8TcGswG1,5L1xVHzn7IaqwxtmA0Z4pA,0ogdfHpftP1jCcTDD0FCY0,1kEsMHskBGccEZg2iqxkEj,5KPBYV9FSX88T65iR0zt9a,17ztPex4W8AT6A6eiNipJ0,3JMQRQsWHRPD5m1FJIH6QA,4VTHITWyaUjHzmwfoXDaB0,587RxTUA1c6JA0g0tvog56
7,13Mf2ZBpfNkgWJowvM5hXh,7eVb9VOmWx4LuRxnagv52t,4MH5viRrL8FFmCK7XYU7p1,3WSp7rcDHOLptPMHnauNZL,2Orb9EImrQhHtUiaXeXhCi,3ellyjQEvUcs5C5vGJ9o2R,1ayvIobYlS9P76iQOxvfSL,0Ec4mfYmYEDh5K3yu8WuHW,7Hzl8tC9YDoWhAqPKubmxI,2OJnEvQa4iXuu27Bow7N7z,3389V9XL6rFml55bfGVmWU,0YB0CNvoPraH3nKDd3YODE,2VFVCndhp8gmtcJ4r6XIRo,4XXV8O94kMneFjqpl9pR9Y,7vBFNgj6XAYYcp6pKuvjp2,1aV0gqqP62CLPhHespP7Vw
8,7BQaRTHk44DkMhIVNcXy2D,02hyyG116hNbmUZeZa8eUH,6yyg0QZO9MVdnGBxZGzwsq,5EN9z8glEyRNboacdxZ03s,5RCPsfzmEpTXMCTNk7wEfQ,7tLKfUEhJw6vAOoZWAmLL5,4iMjxTKlAVPk2uhlxotjyd,1Txr0Rtc7ifjFuMZ7OljPa,0EnSAyMseH22We47ylag4J,6lzVmCvo7WxRfalR2VDlvv,7nGckRfoDs9VAKNStmvJIL,0wej7BQkkXiWgNOC1Ip5Nb,4lncSzeN8WOH2iHEO593iZ,3zeaTZvVBn4V7yQphMAlm0,6UolGvwTTSxm0gsPJinvV4,2rsVe4i85Fhw9sqIyRjNOg
9,049RxG2laEl9U1PGYeIqLV,2j0jrlfp0KsPUHkHigvLrg,3pTGEo5YPIpv9lAqWXoN1z,3A55k8j7w2kgf0xsfVJx8K,6Ms6NDAIIMawqK4tYb0RJD,1WU5uPNUF47WMrn6u5El5D,6GYRW61K0Rd5kdrW62sb2d,5Qb05vMBLVkpqD1jOhtut1,4D4Nq2TAX1BynqVLUam97y,0OnH1Cz0thg4auIy0zIADa,0RDEfBdSE8d8dNwSPYyhim,4ppBpQQNwUbIigW7fqRlKl,6rZHzEvreYJc9dmxwYBYiw,202iLPLHKHX4pNRuZUhDMj,6YDpTzWTqYtVYwRWsNX9hc,41GInuLmXQTunI8miFDm8z


In [ ]:
# predict function



In [ ]:
# From here, if we add both the new dataframe and the original into an SQL database, we can easily
# just run JOIN ON queries to match the song Id's with track_name, artist, and any other info we'd want to display